In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt


In [2]:
FILE_NAME ='data_source/data.csv'

## Load Data

In [29]:
data ={'review':[],'label':[]}
with open(FILE_NAME,'r') as f:
    for line in f:
        fields = line.split('||')
        data['review'].append(fields[0])
        data['label'].append(fields[1].rstrip())
data = pd.DataFrame(data)
data = data[1:]
   

In [41]:
data.shape

(221160, 2)

## Remove Corrupted Labels

In [42]:
data = data[data['label'] != '|neg']
data = data[data['label'] != '|pos']
data = data[data['label'] != 'd initially,but the young lady who managed the desk was masterful in correcting the issue.']

In [43]:
len(data['label'].unique())

2

In [53]:
print("Number of non unique reviews: ",data.shape[0] - len(data['review'].unique()))

Number of non unique reviews:  0


## Remove Duplicates

In [52]:
data = data.drop_duplicates(['review'])

In [54]:
data.shape

(220192, 2)

## Data Exploration

In [59]:
data[data['label'] == 'pos'].head()

,label,review
1,pos,"Cozy hotel with excellent location (a quiet, b..."
2,pos,Friendly staff. Room very big and comfortable...
3,pos,Location is very convenient. Atmosphere of a f...
4,pos,"Everything was ok, main thing going for this h..."
5,pos,Excellent accommodation good location attentiv...


In [58]:
data[data['label'] == 'neg'].head()

,label,review
110318,neg,I'd have to make it up...
110319,neg,Staff were friendly but not particularly proac...
110320,neg,Only moan would be shower curtain really needs...
110321,neg,"Unable to drink water in room, had to get wate..."
110322,neg,They order a taxi transfer for you and add 20 ...
